In [1]:
import requests
import time
import os

In [4]:
# All documentation is here: https://us-fcc.app.box.com/v/bdc-public-data-api-spec
# Includes info on generating username and API token on pp. 4-5

USERNAME = #####
TOKEN = #####

# Base URL for the FCC API
BASE_URL = "https://broadbandmap.fcc.gov/api/public/map/downloads/listAvailabilityData/{as_of_date}"

# Dates 
as_of_dates = ['2022-06-30', '2023-06-30']

# Providers 
providers = ['T-Mobile USA, Inc.', 'AT&T Inc.', 'Verizon Communications Inc.']

# Technology code
LTE_TECHNOLOGY_CODE = '4G LTE'

SyntaxError: invalid syntax (3196158888.py, line 2)

### Optional 1: Return technology codes available

In [ ]:
# Function to fetch all technology codes

as_of_date = '2023-06-30'

def list_technology_codes(as_of_date):
    url = BASE_URL.format(as_of_date=as_of_date)
    
    headers = {
        'username': USERNAME,
        'hash_value': TOKEN
    }

    # Send the request to FCC API
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        
        # Check if data exists in the response
        if isinstance(data, dict) and 'data' in data:
            technology_codes = {record.get('technology_code_desc') for record in data['data'] if record.get('technology_code_desc')}
            return technology_codes
        else:
            print(f"No data found for {as_of_date}.")
            return set()
    else:
        print(f"Failed to get data, status code: {response.status_code}")
        return set()

# Main script
if __name__ == "__main__":
    technology_codes = list_technology_codes(as_of_date)
    
    # Print all unique technology codes
    print("Technology codes:")
    for code in technology_codes:
        print(code)

### Optional 2: Return providers available

In [ ]:
# Function to fetch all provider IDs and names

as_of_date = '2023-06-30'

def list_providers(as_of_date):
    url = BASE_URL.format(as_of_date=as_of_date)
    
    headers = {
        'username': USERNAME,
        'hash_value': TOKEN
    }

    # Send the request to FCC API
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        
        # Check if data exists in the response
        if isinstance(data, dict) and 'data' in data:
            providers = {f"{record.get('provider_id')}, {record.get('provider_name')}" 
                         for record in data['data'] 
                         if record.get('provider_id') and record.get('provider_name')}
            return providers
        else:
            print(f"No data found for {as_of_date}.")
            return set()
    else:
        print(f"Failed to get data, status code: {response.status_code}")
        return set()

# Main script
if __name__ == "__main__":
    providers = list_providers(as_of_date)
    
    # Print all provider_id, provider_name pairs
    print("Provider ID, Provider Name:")
    for provider in providers:
        print(provider)

### Optional 3: Print names of shapefiles that meet criteria

In [ ]:
# Criteria
as_of_dates = ['2022-06-30', '2023-06-30']
provider_names = ['Verizon Communications Inc.', 'T-Mobile USA, Inc.', 'AT&T Inc.']
technology_code = '4G LTE'

# Function to print names of shapefiles that meet criteria
def list_shapefiles(as_of_date, provider_name, technology_code):
    url = BASE_URL.format(as_of_date=as_of_date)
    
    headers = {
        'username': USERNAME,
        'hash_value': TOKEN
    }

    # Send the request to FCC API
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()

        # Check if data exists in the response
        if isinstance(data, dict) and 'data' in data:
            for record in data['data']:
                record_provider_name = record.get('provider_name')
                technology_code_desc = record.get('technology_code_desc')
                file_type = record.get('file_type')

                # Check if provider name matches, technology code is '4G LTE', and file type is 'gis'
                if record_provider_name == provider_name and technology_code_desc == technology_code and file_type == 'gis':
                    file_name = record.get('file_name')
                    print(file_name)  # Only print the shapefile name

# Main script
if __name__ == "__main__":
    for as_of_date in as_of_dates:  # Loop through dates
        for provider_name in provider_names:  # Loop through providers
            list_shapefiles(as_of_date, provider_name, technology_code)


### Check against files that meet criteria - download without duplicates

In [3]:
# Criteria, duplicated in case I don't run cell above
as_of_dates = ['2022-06-30', '2023-06-30']
provider_names = ['Verizon Communications Inc.', 'T-Mobile USA, Inc.', 'AT&T Inc.']
technology_code = '4G LTE'

# Base URL for downloading files based on the API documentation
download_base_url = 'https://broadbandmap.fcc.gov/api/public/map/downloads/downloadFile/availability'

# Directory to save the shapefiles
output_dir = 'D:\ECHO\FCC\API_Downloads'

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to download the shapefile and save it with retry logic
def download_shapefile(file_name, file_url, retries=3):
    file_path = os.path.join(output_dir, file_name)
    
    # Skip downloading if the file already exists
    if os.path.exists(file_path):
        print(f"File already exists: {file_name}, skipping download.")
        return

    attempt = 0
    while attempt < retries:
        try:
            # Download the file
            response = requests.get(file_url, headers={'username': USERNAME, 'hash_value': TOKEN}, stream=True)
            
            # Respect the rate limit: wait 6 seconds after each download request
            time.sleep(6)
            
            if response.status_code == 200:
                # Save the file
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:  # Filter out keep-alive chunks
                            f.write(chunk)
                print(f"Downloaded: {file_name}")
                return  # Download successful, exit the function
            else:
                print(f"Failed to download {file_name}, status code: {response.status_code}")
        except (ChunkedEncodingError, ConnectionError, Timeout) as e:
            attempt += 1
            print(f"Error downloading {file_name}: {e}. Retrying {attempt}/{retries}...")
            time.sleep(5)  # Wait 5 seconds before retrying
    
    print(f"Failed to download {file_name} after {retries} retries.")

# Function to list and download missing shapefiles
def list_and_download_shapefiles(as_of_date, provider_name, technology_code):
    url = BASE_URL.format(as_of_date=as_of_date)
    
    headers = {
        'username': USERNAME,
        'hash_value': TOKEN
    }

    # Send the request to FCC API
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()

        # Check if data exists in the response
        if isinstance(data, dict) and 'data' in data:
            for record in data['data']:
                record_provider_name = record.get('provider_name')
                technology_code_desc = record.get('technology_code_desc')
                file_type = record.get('file_type')

                # Check if provider name matches, technology code is '4G LTE', and file type is 'gis'
                if record_provider_name == provider_name and technology_code_desc == technology_code and file_type == 'gis':
                    file_name = record.get('file_name')
                    file_id = record.get('file_id')  # Extract the file_id from the response

                    # Print the shapefile name and check if it's already downloaded
                    print(f"Checking: {file_name}")
                    
                    # Construct the download URL using file_id and file_type (1 for ESRI Shapefile)
                    if file_id:
                        file_url = f"{download_base_url}/{file_id}/1"
                        download_shapefile(file_name, file_url)  # Download if not already present
                    else:
                        print(f"No file ID found for {file_name}")
        else:
            print(f"No data field found in the response for {as_of_date}.")
    else:
        print(f"Failed to get data for {as_of_date}, status code: {response.status_code}, Response Text: {response.text}")

# Main script to list files and check if they're already downloaded
if __name__ == "__main__":
    for as_of_date in as_of_dates:  # Loop through dates
        for provider_name in provider_names:  # Loop through providers
            list_and_download_shapefiles(as_of_date, provider_name, technology_code)


Checking: bdc_46_131425_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_46_131425_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_55_131425_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_55_131425_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_20_131425_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_20_131425_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_31_131425_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_31_131425_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_32_131425_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_32_131425_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_17_131425_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_17_131425_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_24_131425_4GLTE_mobile_broadband

Checking: bdc_54_130077_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_54_130077_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_24_130077_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_24_130077_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_40_130077_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_40_130077_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_49_130077_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_49_130077_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_06_130077_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_06_130077_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_23_130077_4GLTE_mobile_broadband_h3_J22_10may2024
File already exists: bdc_23_130077_4GLTE_mobile_broadband_h3_J22_10may2024, skipping download.
Checking: bdc_35_130077_4GLTE_mobile_broadband

Checking: bdc_33_130403_4GLTE_mobile_broadband_h3_J23_01sep2024
File already exists: bdc_33_130403_4GLTE_mobile_broadband_h3_J23_01sep2024, skipping download.
Checking: bdc_38_130403_4GLTE_mobile_broadband_h3_J23_01sep2024
File already exists: bdc_38_130403_4GLTE_mobile_broadband_h3_J23_01sep2024, skipping download.
Checking: bdc_13_130403_4GLTE_mobile_broadband_h3_J23_01sep2024
File already exists: bdc_13_130403_4GLTE_mobile_broadband_h3_J23_01sep2024, skipping download.
Checking: bdc_30_130403_4GLTE_mobile_broadband_h3_J23_01sep2024
File already exists: bdc_30_130403_4GLTE_mobile_broadband_h3_J23_01sep2024, skipping download.
Checking: bdc_24_130403_4GLTE_mobile_broadband_h3_J23_01sep2024
File already exists: bdc_24_130403_4GLTE_mobile_broadband_h3_J23_01sep2024, skipping download.
Checking: bdc_35_130403_4GLTE_mobile_broadband_h3_J23_01sep2024
File already exists: bdc_35_130403_4GLTE_mobile_broadband_h3_J23_01sep2024, skipping download.
Checking: bdc_51_130403_4GLTE_mobile_broadband

Downloaded: bdc_30_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Checking: bdc_29_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Downloaded: bdc_29_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Checking: bdc_49_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Downloaded: bdc_49_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Checking: bdc_38_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Downloaded: bdc_38_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Checking: bdc_17_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Downloaded: bdc_17_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Checking: bdc_05_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Downloaded: bdc_05_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Checking: bdc_39_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Downloaded: bdc_39_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Checking: bdc_35_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Downloaded: bdc_35_130077_4GLTE_mobile_broadband_h3_J23_01sep2024
Checking: bdc_37_130077_